# Polytech SI3 - Données numériques: classification de sons
Diane Lingrand
2022-2023


In [3]:
#chargement des librairies
import matplotlib.pyplot as plt
import numpy as np 
import librosa
import librosa.display
import IPython.display as ipd

In [4]:
import glob

In [5]:
basedir="/home/tsukoyachi/Documents/Ecole/SI3/ComputerScience/TD/S6/Données Numériques/TD4/"
# à adapter 
#attention, sous windows, il faut changer en "C:\\Users\\monNom\\monCoursPrefere\\"
classes = ["cat","dog","bird"] 
nbClasses = len(classes)

for cl in classes:
    listSons = glob.glob(basedir+cl+"/*.wav")
    print(cl,len(listSons))

cat 1733
dog 1746
bird 1731


## algorithme kNN

In [6]:
#knn algo
def voteKnn(xtrain, ytrain, nKnn, newData):
    
    classWhenAmbiguity = 2

    d = []
    for PX in xtrain:
        d.append(np.sqrt(np.power(PX[0] - newData[0], 2) + np.power(PX[1] - newData[1], 2) + np.power(PX[2] - newData[2], 2)))

    r = np.argsort(d)

    # nKNN smallest distances indices
    nei1 = r[0:nKnn]

    # nKNN closest data
    Xn1 = []
    yn1 = []
    for ji in nei1:
        Xn1.append(xtrain[ji])
        yn1.append(ytrain[ji])

    cl, co = np.unique(yn1, return_counts=True)
    nbOfMC = np.count_nonzero(co == np.max(co))  # Permet de récupérer le nombre de classe qui ont le nombre d'occurrence max du tableau
    if nbOfMC > 1:
        classe1 = classWhenAmbiguity
    else:
        maxOccIndex1 = np.argmax(co)  # renvoie l'indice de l'occurrence la plus élevée
        classe1 = cl[maxOccIndex1]  # on récupère la classe correspondant à cette occurrence la plus élevée, autrement dit on récupère la classe ayant le nombre d'occurrence le plus élevé, grâce à leur indice identique (dans les tableaux classes et counts, une classe à un indice précis va trouver son occurrence au même indice dans le tableau counts)
    return classe1


## Représentation des sons par 1 MFCC

In [7]:
# loading train dataset
nb=200 # for each class
i = 0
clNumber = 0
yTrain = []
Xtrain = np.empty(shape=(nb*nbClasses, 13), dtype=float)
for cl in classes:
    listSons = glob.glob(basedir+cl+"/*.wav")
    for s in listSons[:nb]:
        (sig,rate) = librosa.load(s)
        mfcc_feat = librosa.feature.mfcc(y=sig,sr=rate,n_mfcc=13, hop_length=len(sig)+1)
        Xtrain[i] = mfcc_feat.reshape(13)
        i += 1
    yTrain += [clNumber]*nb
    clNumber += 1

yTrain = np.array(yTrain)

In [8]:
## loading test dataset
i = 0
clNumber = 0
yTest = []
Xtest = np.empty(shape=(nb*nbClasses, 13), dtype=float)
for cl in classes:
    listSons = glob.glob(basedir+cl+"/*.wav")
    for s in listSons[-nb:]:
    #print("###",s,"###")
        (sig,rate) = librosa.load(s)
        mfcc_feat = librosa.feature.mfcc(y=sig,sr=rate,n_mfcc=13, hop_length=len(sig)+1)    
        Xtest[i] = mfcc_feat.reshape(13)
        i += 1
    yTest += [clNumber]*nb
    clNumber += 1

yTest = np.array(yTest)

## Représentation des sons par la moyenne des MFCCs (longueur standard)

In [9]:
# loading train dataset
nb=200 # for each class
i = 0
clNumber = 0
yTrain = []
Xtrain = np.empty(shape=(nb*nbClasses, 13), dtype=float)
for cl in classes:
    listSons = glob.glob(basedir+cl+"/*.wav")
    for s in listSons[:nb]:
        (sig,rate) = librosa.load(s)
        mfcc_feat = librosa.feature.mfcc(y=sig,sr=rate,n_mfcc=13)
        Xtrain[i] = np.mean(mfcc_feat, axis=1)
        i += 1
    yTrain += [clNumber]*nb
    clNumber += 1

yTrain = np.array(yTrain)

In [10]:
## loading test dataset
i = 0
clNumber = 0
yTest = []
Xtest = np.empty(shape=(nb*nbClasses, 13), dtype=float)
for cl in classes:
    listSons = glob.glob(basedir+cl+"/*.wav")
    for s in listSons[-nb:]:
    #print("###",s,"###")
        (sig,rate) = librosa.load(s)
        mfcc_feat = librosa.feature.mfcc(y=sig,sr=rate,n_mfcc=13)
        Xtest[i] = np.mean(mfcc_feat, axis=1)
        i += 1
    yTest += [clNumber]*nb
    clNumber += 1

yTest = np.array(yTest)

## Classification par kNN

**Question 1**: Pour chaque représentation des fichiers sons, calculez les prédictions d'une classification par kNN. Pour cela, il faudra tester plusieurs valeurs de k.

In [11]:
numberNN = 5

In [14]:
#predictions
predTrain = [voteKnn(Xtrain,yTrain,numberNN,i) for i in Xtrain]
predTest = [voteKnn(Xtest,yTest,numberNN,i) for i in Xtrain]

**Question 2**: Calculez les matrices de confusion, les métriques de précision 'accuracy' et de score F1 pour les données d'apprentissage et de test. Pour cela, écrire 3 fonctions: <ul> <li> une fonction qui affiche une matrice de confusion</li> <li>une fonction qui calcule la précision (accuracy)</li><li>une fonction qui calcule le score F1 d'une classe</li></ul>

In [60]:
def printConfusionMatrix(matrixContent) :
    classesName = np.unique(matrixContent)
    nbClasse = len(classesName)
    res = np.zeros((nbClasse,nbClasse))
    for i in range (0, nbClasse) :
        for y in range (0, 200) :
            res[i][matrixContent[y + 200*i]] += 1
        print("For data of classe "+str(i)+" :")
        for z in range (0, nbClasse) :
            print(str(res[i][z]) + " element of classe "+str(z))
        print("-------------------------")
    return res

def accuracy(confusionMatrix) :
    nbClasse = np.shape(confusionMatrix)[0]
    correct = 0
    total = 0
    for i in range (0,nbClasse) :
        for y in range (0,nbClasse) :
            if i == y :
                correct += confusionMatrix[i][y]
            total += confusionMatrix[i][y]
    accuracy = correct/total
    print(accuracy)
    return accuracy

def f1(confusionMatrix) :
    nbClasse = np.shape(confusionMatrix)[0]
    vp = 0
    total = 0
    for i in range (0,nbClasse) :
        for y in range (0,nbClasse) :
            if i == y :
                vp += confusionMatrix[i][y]
            total += confusionMatrix[i][y]

**Question 3:** Affichez la courbe de la précision (accuracy) en fonction du k de kNN pour les données d'apprentissage et pour les données de test.

In [63]:
res = []
for n in range (0,50) :
    predTrain = [voteKnn(Xtrain,yTrain,n,i) for i in Xtrain]
    predTest = [voteKnn(Xtest,yTest,n,i) for i in Xtrain]
    confusionMatrix = printConfusionMatrix(predTest)
    accuracy = accuracy(confusionMatrix)
    res.append(accuracy)
plt.plot(res,n)

ValueError: zero-size array to reduction operation maximum which has no identity

**Question 4:** Parmi les 2 représentations proposées, laquelle vous parait la meilleure?

In [ ]:
#ici